In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split,StratifiedKFold,GridSearchCV

from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

import warnings
warnings.filterwarnings('ignore')

In [2]:
image=pd.read_csv("Image_Segmention.csv")
image.head()

,Class,region.centroid.col,region.centroid.row,region.pixel.count,short.line.density.5,short.line.density.2,vedge.mean,vegde.sd,hedge.mean,hedge.sd,intensity.mean,rawred.mean,rawblue.mean,rawgreen.mean,exred.mean,exblue.mean,exgreen.mean,value.mean,saturation.mean,hue-mean
0,BRICKFACE,188,133,9,0.0,0.0,0.333333,0.266667,0.500000,0.077778,6.666666,8.333334,7.777778,3.888889,5.000000,3.333333,-8.333333,8.444445,0.538580,-0.924817
1,BRICKFACE,105,139,9,0.0,0.0,0.277778,0.107407,0.833333,0.522222,6.111111,7.555555,7.222222,3.555556,4.333334,3.333333,-7.666666,7.555555,0.532628,-0.965946
2,BRICKFACE,34,137,9,0.0,0.0,0.500000,0.166667,1.111111,0.474074,5.851852,7.777778,6.444445,3.333333,5.777778,1.777778,-7.555555,7.777778,0.573633,-0.744272
3,BRICKFACE,39,111,9,0.0,0.0,0.722222,0.374074,0.888889,0.429629,6.037037,7.000000,7.666666,3.444444,2.888889,4.888889,-7.777778,7.888889,0.562919,-1.175773
4,BRICKFACE,16,128,9,0.0,0.0,0.500000,0.077778,0.666667,0.311111,5.555555,6.888889,6.666666,3.111111,4.000000,3.333333,-7.333334,7.111111,0.561508,-0.985811


In [3]:
lbl=LabelEncoder()
image['Class']=lbl.fit_transform(image['Class'])
image.head()

,Class,region.centroid.col,region.centroid.row,region.pixel.count,short.line.density.5,short.line.density.2,vedge.mean,vegde.sd,hedge.mean,hedge.sd,intensity.mean,rawred.mean,rawblue.mean,rawgreen.mean,exred.mean,exblue.mean,exgreen.mean,value.mean,saturation.mean,hue-mean
0,0,188,133,9,0.0,0.0,0.333333,0.266667,0.500000,0.077778,6.666666,8.333334,7.777778,3.888889,5.000000,3.333333,-8.333333,8.444445,0.538580,-0.924817
1,0,105,139,9,0.0,0.0,0.277778,0.107407,0.833333,0.522222,6.111111,7.555555,7.222222,3.555556,4.333334,3.333333,-7.666666,7.555555,0.532628,-0.965946
2,0,34,137,9,0.0,0.0,0.500000,0.166667,1.111111,0.474074,5.851852,7.777778,6.444445,3.333333,5.777778,1.777778,-7.555555,7.777778,0.573633,-0.744272
3,0,39,111,9,0.0,0.0,0.722222,0.374074,0.888889,0.429629,6.037037,7.000000,7.666666,3.444444,2.888889,4.888889,-7.777778,7.888889,0.562919,-1.175773
4,0,16,128,9,0.0,0.0,0.500000,0.077778,0.666667,0.311111,5.555555,6.888889,6.666666,3.111111,4.000000,3.333333,-7.333334,7.111111,0.561508,-0.985811


In [4]:
X=image.drop('Class',axis=1)
y=image['Class']

In [5]:
svm=SVC(probability=True,random_state=23)
lr=LogisticRegression()
nb=GaussianNB()

In [6]:
voting=VotingClassifier(estimators=[('SVM',svm),('LR',lr),('NB',nb)],voting='soft')

In [7]:
kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=23)
params = {'SVM__gamma':['scale','auto'],
          'SVM__C':np.linspace(0.001,5,5),
          'LR__multi_class':['auto', 'ovr', 'multinomial'],
          'LR__penalty':['l1','l2','elasticnet',None]}
gcv=GridSearchCV(voting,param_grid=params,cv=kfold,scoring='neg_log_loss')
gcv.fit(X,y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=VotingClassifier(estimators=[('SVM',
                                                     SVC(probability=True,
                                                         random_state=23)),
                                                    ('LR',
                                                     LogisticRegression()),
                                                    ('NB', GaussianNB())],
                                        voting='soft'),
             param_grid={'LR__multi_class': ['auto', 'ovr', 'multinomial'],
                         'LR__penalty': ['l1', 'l2', 'elasticnet', None],
                         'SVM__C': array([1.00000e-03, 1.25075e+00, 2.50050e+00, 3.75025e+00, 5.00000e+00]),
                         'SVM__gamma': ['scale', 'auto']},
             scoring='neg_log_loss')

In [8]:
print("best parameter :",gcv.best_params_)
print("best score :",gcv.best_score_)

best parameter : {'LR__multi_class': 'auto', 'LR__penalty': None, 'SVM__C': 5.0, 'SVM__gamma': 'scale'}
best score : -0.4558017806092759


## Only SVM

In [9]:
params={'gamma':['scale','auto'],
          'C':np.linspace(0.001,5,5)}
gcv=GridSearchCV(svm,param_grid=params,cv=kfold,scoring='neg_log_loss')
gcv.fit(X,y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=SVC(probability=True, random_state=23),
             param_grid={'C': array([1.00000e-03, 1.25075e+00, 2.50050e+00, 3.75025e+00, 5.00000e+00]),
                         'gamma': ['scale', 'auto']},
             scoring='neg_log_loss')

In [10]:
print("best parameter :",gcv.best_params_)
print("best score :",gcv.best_score_)

best parameter : {'C': 5.0, 'gamma': 'scale'}
best score : -0.622653795419567


## Only LR

In [11]:
params = {'multi_class':['auto', 'ovr', 'multinomial'],
          'penalty':['l1','l2','elasticnet',None]}
gcv=GridSearchCV(lr,param_grid=params,cv=kfold,scoring='neg_log_loss')
gcv.fit(X,y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=LogisticRegression(),
             param_grid={'multi_class': ['auto', 'ovr', 'multinomial'],
                         'penalty': ['l1', 'l2', 'elasticnet', None]},
             scoring='neg_log_loss')

In [12]:
print("best parameter :",gcv.best_params_)
print("best score :",gcv.best_score_)

best parameter : {'multi_class': 'ovr', 'penalty': 'l2'}
best score : -0.6771400409941282


## Only NB

In [13]:
params = {}
gcv=GridSearchCV(nb,param_grid=params,cv=kfold,scoring='neg_log_loss')
gcv.fit(X,y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=GaussianNB(), param_grid={}, scoring='neg_log_loss')

In [14]:
print("best parameter :",gcv.best_params_)
print("best score :",gcv.best_score_)

best parameter : {}
best score : -3.795338814659644


## weighted

In [15]:
voting=VotingClassifier(estimators=[('SVM',svm),('LR',lr),('NB',nb)],voting='soft',
                        weights=[0.5,0.4,0.1])

In [16]:
kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=23)
params = {'SVM__gamma':['scale','auto'],
          'SVM__C':np.linspace(0.001,5,5),
          'LR__multi_class':['auto', 'ovr', 'multinomial'],
          'LR__penalty':['l1','l2','elasticnet',None]}
gcv=GridSearchCV(voting,param_grid=params,cv=kfold,scoring='neg_log_loss')
gcv.fit(X,y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=VotingClassifier(estimators=[('SVM',
                                                     SVC(probability=True,
                                                         random_state=23)),
                                                    ('LR',
                                                     LogisticRegression()),
                                                    ('NB', GaussianNB())],
                                        voting='soft',
                                        weights=[0.5, 0.4, 0.1]),
             param_grid={'LR__multi_class': ['auto', 'ovr', 'multinomial'],
                         'LR__penalty': ['l1', 'l2', 'elasticnet', None],
                         'SVM__C': array([1.00000e-03, 1.25075e+00, 2.50050e+00, 3.75025e+00, 5.00000e+00]),
                         'SVM__gamma': ['scale', 'auto']},
             scoring='neg_log_loss')

In [17]:
print("best parameter :",gcv.best_params_)
print("best score :",gcv.best_score_)

best parameter : {'LR__multi_class': 'auto', 'LR__penalty': None, 'SVM__C': 5.0, 'SVM__gamma': 'scale'}
best score : -0.4530115367788962


## Unweighted(using DTC)

In [18]:
svm=SVC(probability=True,random_state=23)
lr=LogisticRegression()
dtc=DecisionTreeClassifier()

In [19]:
voting=VotingClassifier(estimators=[('SVM',svm),('LR',lr),('DTC',dtc)],voting='soft')

In [20]:
kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=23)
params = {'SVM__gamma':['scale','auto'],
          'SVM__C':np.linspace(0.001,5,5),
          'LR__multi_class':['auto', 'ovr', 'multinomial'],
          'LR__penalty':['l1','l2','elasticnet',None],
          'DTC__max_depth':[2,3,5,10]}
gcv=GridSearchCV(voting,param_grid=params,cv=kfold,scoring='neg_log_loss')
gcv.fit(X,y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=VotingClassifier(estimators=[('SVM',
                                                     SVC(probability=True,
                                                         random_state=23)),
                                                    ('LR',
                                                     LogisticRegression()),
                                                    ('DTC',
                                                     DecisionTreeClassifier())],
                                        voting='soft'),
             param_grid={'DTC__max_depth': [2, 3, 5, 10],
                         'LR__multi_class': ['auto', 'ovr', 'multinomial'],
                         'LR__penalty': ['l1', 'l2', 'elasticnet', None],
                         'SVM__C': array([1.00000e-03, 1.25075e+00, 2.50050e+00, 3.75025e+00, 5.00000e+00]),
                         'SVM__gamma': ['scale', 'auto']},
             scoring='neg_log_loss')

In [21]:
print("best parameter :",gcv.best_params_)
print("best score :",gcv.best_score_)

best parameter : {'DTC__max_depth': 10, 'LR__multi_class': 'multinomial', 'LR__penalty': None, 'SVM__C': 1.2507499999999998, 'SVM__gamma': 'scale'}
best score : -0.3512813302669295


## Only DTC

In [22]:
kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=23)
params = {'max_depth':[2,3,5,10]}
gcv=GridSearchCV(dtc,param_grid=params,cv=kfold,scoring='neg_log_loss')
gcv.fit(X,y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [2, 3, 5, 10]}, scoring='neg_log_loss')

In [23]:
print("best parameter :",gcv.best_params_)
print("best score :",gcv.best_score_)

best parameter : {'max_depth': 3}
best score : -1.3042564401140002


## Weighted with dtc

In [24]:
voting=VotingClassifier(estimators=[('SVM',svm),('LR',lr),('DTC',dtc)],voting='soft',
                        weights=[0.45,0.35,0.2])

In [25]:
kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=23)
params = {'SVM__gamma':['scale','auto'],
          'SVM__C':np.linspace(0.001,5,5),
          'LR__multi_class':['auto', 'ovr', 'multinomial'],
          'LR__penalty':['l1','l2','elasticnet',None],
          'DTC__max_depth':[2,3,5,10]}
gcv=GridSearchCV(voting,param_grid=params,cv=kfold,scoring='neg_log_loss')
gcv.fit(X,y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=VotingClassifier(estimators=[('SVM',
                                                     SVC(probability=True,
                                                         random_state=23)),
                                                    ('LR',
                                                     LogisticRegression()),
                                                    ('DTC',
                                                     DecisionTreeClassifier())],
                                        voting='soft',
                                        weights=[0.45, 0.35, 0.2]),
             param_grid={'DTC__max_depth': [2, 3, 5, 10],
                         'LR__multi_class': ['auto', 'ovr', 'multinomial'],
                         'LR__penalty': ['l1', 'l2', 'elasticnet', None],
                         'SVM__C': array([1.00000e-03, 1.25075e+00, 2.50050e+00, 3.75025e+00, 5.00000e+00]),
                         'SVM__gamma': ['scale', 'auto']},
             scoring='neg_log_loss')

In [26]:
print("best parameter :",gcv.best_params_)
print("best score :",gcv.best_score_)

best parameter : {'DTC__max_depth': 10, 'LR__multi_class': 'multinomial', 'LR__penalty': 'l2', 'SVM__C': 5.0, 'SVM__gamma': 'scale'}
best score : -0.3970337255019244
